<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Meus-Codigos/blob/main/Ler_Verbatins_ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
import pandas as pd
import torch

In [2]:
# Verifica se há GPU disponível
device = 0 if torch.cuda.is_available() else -1

In [3]:
# Carrega pipeline com modelo multilíngue
analisador = pipeline("sentiment-analysis",
                      model="nlptown/bert-base-multilingual-uncased-sentiment",
                      device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Lê os comentários
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Arquivos_Ler/Verbatins.xlsx")

In [ ]:
df.head(10)

In [ ]:
# Função para aplicar análise
def analisar_sentimento(texto):
    resultado = analisador(str(texto))[0]
    label = resultado['label']
    score = resultado['score']

    # Converte label para categoria
    estrelas = int(label[0])
    if estrelas <= 2:
        categoria = "Negativo"
    elif estrelas == 3:
        categoria = "Neutro"
    else:
        categoria = "Positivo"

    return pd.Series([label, score, categoria])

In [ ]:
# Aplica ao DataFrame
df[["Label_Modelo", "Confianca", "Categoria"]] = df["Comentário"].apply(analisar_sentimento)

In [ ]:
df.head(10)

In [ ]:
# Exportar o resultado
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Arquivos_Ler/comentarios_analisados_vader.csv", index=False)